# Face Detection

In [44]:
#import necessary modules

import numpy as np
import pandas as pd
import cv2
import os

## Load images from dir


In [45]:
#images path
img_dir = '/workspaces/faceid/train_images'
# Path to the folder containing test images
test_img_dir = '/workspaces/faceid/test_images'
# Create a mapping from folder names to integer labels
label_map = {'wmaathai': 1, 'others': 0}

## Train FaceRecognizer Model

In [46]:
# initialize empty lists for training images and image-labels
faces = []
ids = []

In [47]:
# train FaceRecognizer model
for face_id in os.listdir(img_dir):
    face_path = os.path.join(img_dir, face_id)
    
    if os.path.isdir(face_path): # Check if it's a directory (person's folder)
        # Check if the folder name is in the label_map
        if face_id in label_map:
            label = label_map[face_id]
        else:
            continue  # Skip folders not in the label map

        # process folder images
        for img_label in os.listdir(face_path):
            img_path = os.path.join(face_path, img_label)
            
            img = cv2.imread(img_path)

            #skip invalid images
            if img is None:
                print(f"Failed to load image {img_path}")
                continue

            # convert image to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # load Haar CascadeClasifier and detect faces
            face_cascade = cv2.CascadeClassifier('/workspaces/faceid/haarcascades/haarcascade_frontalface_default.xml')
            faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5) #minSize=(40, 40)

            print(f"Detected {len(faces_detected)} face(s) in {img_path}")

            # Process each face detected
            for (x, y, w, h) in faces_detected:
                face = gray_img[y:y+h, x:x+w]  # Extract face region
                faces.append(face)  # Add the face to the list
                ids.append(int(label))  # Add the corresponding label (ID)


Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/8.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/22.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/11.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/13.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/27.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/9.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/16.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/24.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/21.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/23.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/5.jpg
Detected 1 face(s) in /workspaces/faceid/train_images/wmaathai/3.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/14.jpg
Detected 0 face(s) in /workspaces/faceid/train_images/wmaathai/26.jpg
Detected 0 face(s) in /w

## Face Detection
-consider if a face is not detected

In [48]:
# processing images
"""
(x, y) is the top-left corner of the rectangle
(w, h) is the width and height of the rectangle
"""
for (x, y, w, h) in faces_detected: # face bounding box
    face = gray_img[y:y+h, x:x+w] # extract face from gray_img
    faces.append(face) # add extracted face to list
    ids.append(int(face_id))  # Label with the id (folder name)
    #label, confidence = recognizer.predict(face_region)

In [49]:
"""
print(faces)
print(ids)
# Check if faces and ids are empty
if len(faces) == 0 or len(ids) == 0:
    print("No faces detected or empty data.")
else:
    print(f"Training data: {len(faces)} faces with {len(ids)} labels.")
"""

'\nprint(faces)\nprint(ids)\n# Check if faces and ids are empty\nif len(faces) == 0 or len(ids) == 0:\n    print("No faces detected or empty data.")\nelse:\n    print(f"Training data: {len(faces)} faces with {len(ids)} labels.")\n'

In [50]:
# Train the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(ids))


In [51]:
# Save the trained model
recognizer.save('/workspaces/faceid/trained/trained_model.yml')
print("Model trained and saved successfully.")


Model trained and saved successfully.


# Testing

In [52]:
# Load the trained model for face recognition
recognizer.read('/workspaces/faceid/trained/trained_model.yml')

In [55]:
# Loop through the test images
for test_image_name in os.listdir(test_img_dir):
    test_image_path = os.path.join(test_img_dir, test_image_name)

    test_img = cv2.imread(test_image_path)
    if test_img is None:
        print(f"Failed to load image {test_image_path}")
        continue  # Skip invalid images

    # Convert test image to grayscale for face detection
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the test image
    faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5)

    # Variable to track if a recognized face is found
    is_wmaathai = False
    print(f"Processing image: {test_image_name}")

    # Process each face detected
    for (x, y, w, h) in faces_detected:
        face_region = gray_img[y:y + h, x:x + w]  # Extract the face region

       # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected face with label: {label}, confidence: {confidence}")

        # check if value of recognition confidence is good
        if confidence < 100:  # Low confidence means a good match
            if label == 1:
                is_wmaathai = True
                print(f"{test_image_name} is wmaathai")
            else:
                print(f"{test_image_name} is not wmaathai")
        else:
            # If the confidence is too high, mark the face as unknown
            print(f"{test_image_name} is unknown (confidence too high)")

# If no faces were detected or recognized, output unknown
if len(faces_detected) == 0:
    print(f"No faces detected in {test_image_name}")

        

Processing image: 8.jpg
Processing image: 11.jpg
Processing image: 13.jpg
  Detected face with label: 0, confidence: 86.99078925398025
13.jpg is not wmaathai
Processing image: 9.jpg
Processing image: 16.jpg
  Detected face with label: 0, confidence: 107.71763313927563
16.jpg is unknown (confidence too high)
Processing image: 5.jpg
Processing image: 3.jpg
  Detected face with label: 0, confidence: 67.41206249413666
3.jpg is not wmaathai
Processing image: 14.jpg
  Detected face with label: 0, confidence: 103.99205966192766
14.jpg is unknown (confidence too high)
Processing image: 1.jpg
Processing image: 4.jpg
  Detected face with label: 0, confidence: 108.01684593306607
4.jpg is unknown (confidence too high)
Processing image: 15.jpg
Processing image: 20.jpg
Processing image: 7.jpg
Processing image: 2.jpg
Processing image: 12.jpg
Processing image: 10.jpg
  Detected face with label: 0, confidence: 91.24775460385793
10.jpg is not wmaathai
Processing image: 18.jpg
  Detected face with label:

In [57]:
# Draw a rectangle around the face and display the label
        # color = (0, 255, 0)  # Green color for recognized faces    
#cv2.rectangle(test_img, (x, y), (x + w, y + h), color, 2)
            #cv2.putText(test_img, f'Person {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        #else:
            # If the confidence is high, mark the face as unknown
            #cv2.putText(test_img, 'Unknown', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Show the result
    #cv2.imshow(f"Recognized Faces - {test_image_name}", test_img)
    # Save the result to a file instead of showing it
    #cv2.imwrite("recognized_faces.jpg", test_img)

    #cv2.waitKey(0)  # Wait for any key to close the window

#cv2.destroyAllWindows()
